In [1]:
import os
os.chdir('../../scripts/')
from analysis_tools_cython import *

In [2]:
from astropy.io import fits

In [3]:
hdul = fits.open('/storage/astro2/phrdhx/eleanor-lite/s0006/0000/0000/0416/0330/hlsp_gsfc-eleanor-lite_tess_ffi_s0006-0000000004160330_tess_v1.0_lc.fits')

In [5]:
hdul.info()

Filename: /storage/astro2/phrdhx/eleanor-lite/s0006/0000/0000/0416/0330/hlsp_gsfc-eleanor-lite_tess_ffi_s0006-0000000004160330_tess_v1.0_lc.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     219   ()      
  1                1 BinTableHDU     36   993R x 13C   [D, E, E, E, E, J, E, E, E, E, E, J, E]   
  2                1 BinTableHDU     16   15R x 3C   [20D, 20E, 20E]   


In [23]:
hdul[1].data

FITS_rec([(1468.28703014, 0.00525988, 49.77922 , 5.6269012e+01, 0.90781987,      0, 4.6457715e+00, 6.3689213e+01, 7.572987 ,  3.627726  , 225.49693, 11559,  57.823322),
          (1468.30786335, 0.00525976, 51.029617, 5.5852020e+01, 0.90793884,      0, 4.6456146e+00, 6.3689190e+01, 7.563639 ,  3.6321914 , 225.145  , 11560,  59.160995),
          (1468.32869656, 0.00525965, 52.928207, 5.7760674e+01, 0.9083148 ,      0, 4.6453066e+00, 6.3689674e+01, 7.558605 ,  3.6379054 , 225.19772, 11561,  61.47891 ),
          (1468.34952977, 0.00525953, 50.712414, 5.6913532e+01, 0.90889555,      0, 4.6455426e+00, 6.3689499e+01, 7.5631886,  3.6365843 , 225.41685, 11562,  59.042744),
          (1468.37036297, 0.00525941, 50.608624, 5.4734425e+01, 0.9091005 ,      0, 4.6451325e+00, 6.3689987e+01, 7.5760903,  3.6309526 , 225.40106, 11563,  58.89019 ),
          (1468.39119618, 0.00525929, 92.595055, 1.7786004e+18, 0.92753637,    164, 7.1315790e+06, 7.1315790e+06, 8.47312  , -0.50093657, 230.03514, 11564,

In [13]:
hdul2 = fits.open('/storage/astro2/phsqzm/TESS/SPOC_30min/S06/target/0000/0002/7057/7175/hlsp_tess-spoc_tess_phot_0000000270577175-s0006_tess_v1_lc.fits')

In [18]:
list(hdul[0].header.keys())

['SIMPLE',
 'BITPIX',
 'NAXIS',
 'EXTEND',
 'INHERIT',
 'EXTVER',
 'SIMDATA',
 'TELESCOP',
 'INSTRUME',
 'CAMERA',
 'CCD',
 'EXPOSURE',
 'TIMEREF',
 'TASSIGN',
 'TIMESYS',
 'BJDREFI',
 'BJDREFF',
 'TIMEUNIT',
 'TELAPSE',
 'LIVETIME',
 'TSTART',
 'TSTOP',
 'DATE-OBS',
 'DATE-END',
 'DEADC',
 'TIMEPIXR',
 'TIERRELA',
 'BTC_PIX1',
 'BTC_PIX2',
 'BARYCORR',
 'INT_TIME',
 'READTIME',
 'FRAMETIM',
 'NUM_FRM',
 'TIMEDEL',
 'BACKAPP',
 'DEADAPP',
 'VIGNAPP',
 'GAINA',
 'GAINB',
 'GAINC',
 'GAIND',
 'READNOIA',
 'READNOIB',
 'READNOIC',
 'READNOID',
 'NREADOUT',
 'FXDOFF',
 'TMOFST11',
 'MEANBLCA',
 'MEANBLCB',
 'MEANBLCC',
 'MEANBLCD',
 'LVCSA',
 'LVCEA',
 'TVCSA',
 'TVCEA',
 'SCCSA',
 'SCCEA',
 'SCICOLHA',
 'LVCSB',
 'LVCEB',
 'TVCSB',
 'TVCEB',
 'SCCSB',
 'SCCEB',
 'SCICOLHB',
 'LVCSC',
 'LVCEC',
 'TVCSC',
 'TVCEC',
 'SCCSC',
 'SCCEC',
 'SCICOLHC',
 'LVCSD',
 'LVCED',
 'TVCSD',
 'TVCED',
 'SCCSD',
 'SCCED',
 'SCICOLHD',
 'SCIROWS',
 'SCIROWE',
 'VROWS',
 'VROWE',
 'SMROWS',
 'SMROWE',
 'BFRO

## Functions

In [22]:
hdul[0].header['CAMERA']

1

In [ ]:
def import_eleanorlite(file_path,sector: int,clip=3,drop_bad_points=True,ok_flags=[],pipeline='eleanor-lite',return_type='astropy'):

    """
     Function: Imports XRP lightcurve and performs data cleaning.
    
    
    Parameters:
    :file_path (str): The path to the lightcurve files (in .pkl format) to import.
    :sector (int): TESS Sector.
    :clip (float, optional): The sigma value used for sigma clipping. The default value is 3.
    :drop_bad_points (bool, optional): Removes outliers based on non-zero quality flags. The default value is True.
    :ok_flags (list, optional): A list of additional quality flags that are considered acceptable and should not be dropped during the preprocessing. 
        - New flag [14] is the MAD excluded data.
    :return_type (str, optional): Specifies the format of the returned data. 'astropy' returns an astropy Table object, while 'pandas' returns a pandas DataFrame. The default value is 'astropy'.
    
    Returns:
    :data (pd.DataFrame or astropy.table.Table): The preprocessed lightcurve data.
    :info (list): Information about the lightcurve: TIC ID, RA, DEC, TESS magnitude, Camera, and Chip."""




    try:
        hdulist = fits.open(file_path)
    except FileNotFoundError:
        print("Import failed: file not found")
        return
    
    objdata = hdulist[0].header
    scidata = hdulist[1].data

    if (pipeline == 'eleanor') or pipeline == 'eleanor-lite':
        table = Table(scidata)['TIME','CORR_FLUX','PCA_FLUX','QUALITY','FLUX_ERR']
        info = [objdata['OBJECT'],objdata['KEPLERID'],objdata['KEPMAG'],objdata['QUARTER'],objdata['RA_OBJ'],objdata['DEC_OBJ']]

    sec = objdata['SECTOR']
    camera = objdata['CAMERA']
    
    # loading Ethan Kruse bad times
    bad_times = xrpdata.load_bad_times()
    bad_times = bad_times - 2457000
    
    # loading MAD 
    mad_df = xrpdata.load_mad()

    camera = lc[4]
    mad_arr = mad_df.loc[:len(table)-1,f"{sec}-{camera}"]
    sig_clip = sigma_clip(mad_arr,sigma=clip,masked=True)

    # applied MAD cut to keep points within selected sigma
    mad_cut = mad_arr.values < ~sig_clip.mask 
    
    # return indices of values above MAD threshold
    matched_ind = np.where(~mad_cut) # indices of MAD's above threshold

    # Change quality of matched indices to 2**(17-1) (or add 2**(17-1) if existing flag already present)
    table['quality'][matched_ind] += 2**(17-1)
    table['quality'] = table['quality'].astype(np.int32) # int32 set so it can work with `get_quality_indices` function

    # Ethan Kruse bad time mask
    mask = np.ones_like(table['time'], dtype=bool)
    for i in bad_times:
        newchunk = (table['time']<i[0])|(table['time']>i[1])
        mask = mask & newchunk
        
    # Apply Kruse bad mask to table
    table = table[mask]

    if drop_bad_points:
        bad_points = []
        q_ind = get_quality_indices(table['quality'])
    
        for j,q in enumerate(q_ind): # j=index, q=quality
            if j+1 not in ok_flags:
                bad_points += q.tolist()
        table.remove_rows(bad_points)

    # Delete rows containing NaN values. 
    nan_rows = [ i for i in range(len(table)) if
            math.isnan(table[i][2]) or math.isnan(table[i][0]) ] 
    table.remove_rows(nan_rows)

    # Smooth data by deleting overly 'spikey' points.
    spikes = [ i for i in range(1,len(table)-1) if \
            abs(table[i][1] - 0.5*(table[i-1][1]+table[i+1][1])) \
            > 3*abs(table[i+1][1] - table[i-1][1])]

    for i in spikes:
        table[i][1] = 0.5*(table[i-1][1] + table[i+1][1])

    if return_type == 'pandas':

        return table.to_pandas(), lc[0:6]
    else:

        return table, lc[0:6]